- *Sinh viên làm trực tiếp vào file này, thêm code vào những chỗ TODO*
- *Ouput đúng được đính kèm.*

In [102]:
import pandas as pd
import numpy as np
import plotly.express as px

In [103]:
# Đọc Dataframe từ file csv, với các cột như sau
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
babynames_df = pd.read_csv('Data/STATE.CA.TXT',header=None,names=field_names)

In [104]:
# Tính tổng số trẻ được sinh ra trong từng năm
babynames_df.groupby('Year')['Count'].sum().to_frame()


,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26926
...,...
2018,395436
2019,386996
2020,362882


In [105]:
# Năm đầu tiên mỗi tên xuất hiện
babynames_df.groupby('Name')['Year'].min().to_frame()


,Year
Name,
Aadan,2008
Aadarsh,2019
Aaden,2007
Aadhav,2014
Aadhini,2022
...,...
Zymir,2020
Zyon,1999
Zyra,2012


In [106]:
# Đối với mỗi tên, hãy cho biết giá trị Count từng năm lớn nhất là bao nhiêu
babynames_df.groupby('Name')['Count'].max().to_frame()



,Count
Name,
Aadan,7
Aadarsh,6
Aaden,158
Aadhav,8
Aadhini,6
...,...
Zymir,5
Zyon,17
Zyra,16


In [107]:
# Sort dataframe chỉ gồm Female theo năm
f_babynames = babynames_df[babynames_df['Sex'] == 'F'].sort_values(by='Year')
f_babynames


,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
148,CA,F,1910,Merle,9
149,CA,F,1910,Rosalie,9
150,CA,F,1910,Rosie,9
151,CA,F,1910,Teresa,9
...,...,...,...,...,...
237072,CA,F,2022,Johana,22
237073,CA,F,2022,Kit,22
237074,CA,F,2022,Korra,22
237076,CA,F,2022,Lavender,22


In [108]:
# Đoạn code sau được cung cấp sẵn để minh họa bài toán.
# Plot sự thay đổi của 'Jennifer' theo năm
fig = px.line(f_babynames[f_babynames["Name"] == "Jennifer"],
              x = "Year", y = "Count")
fig.update_layout(font_size = 18, 
                  autosize=False, 
                  width=1000, 
                  height=400)

In [109]:
# Hãy cho biết số lượng baby có tên là 'Jennifer' trong năm có count lớn nhất. 
jennifer_max_count = f_babynames[f_babynames["Name"] == "Jennifer"]["Count"].max()
jennifer_max_count


6065

In [110]:
# Hãy cho biết số lượng baby có tên là 'Jennifer' trong năm gần nhất (2022)
jennifer_2022 = f_babynames[(f_babynames["Name"] == "Jennifer") & (f_babynames["Year"] == 2022)]["Count"].sum()
jennifer_2022


114

In [111]:
# Gọi tỷ lệ giữa số lượng Count hiện tại và Count tối đa là RTP (ratio to peak)
# Tính tỉ lệ này cho 'Jennifer' 
jennifer_rtp = jennifer_2022 / jennifer_max_count
jennifer_rtp


0.018796372629843364

In [112]:
# Tính tỉ lệ trên cho tất cả các name, sort theo RTP

# Tính RTP cho tất cả các tên
rtp_data = []

for name in babynames_df['Name'].unique():
    df_name = babynames_df[babynames_df['Name'] == name]
    max_count = df_name['Count'].max()
    latest_year = df_name['Year'].max()
    cur_count = df_name[df_name['Year'] == latest_year]['Count'].sum()
    rtp = cur_count / max_count if max_count > 0 else 0
    rtp_data.append({'Name': name, 'RTP': rtp})

# Chuyển sang DataFrame và sắp xếp theo RTP
rtp_df = pd.DataFrame(rtp_data).sort_values(by='RTP').query('RTP <= 1')
rtp_df



,Name,RTP
1392,Debra,0.001260
1277,Debbie,0.002815
157,Carol,0.003180
1587,Tammy,0.003249
263,Susan,0.003305
...,...,...
12600,Avaree,1.000000
12602,Bethenny,1.000000
12358,Sarenity,1.000000
12603,Brynlie,1.000000


In [113]:
# Đoạn code sau được cung cấp để minh họa bài toán.
# Plot sự thay đổi của tên 'Debra'
def plot_name(*names):
    fig = px.line(f_babynames[f_babynames["Name"].isin(names)], 
                  x = "Year", y = "Count", color="Name",
                  title=f"Popularity for: {names}")
    fig.update_layout(font_size = 18, 
                  autosize=False, 
                  width=1000, 
                  height=400)
    return fig

plot_name("Debra")

In [114]:
# Dựa vào phần trên, hãy plot sự thay đổi của 10 tên suy giảm nhiều nhất
top_declined_names = rtp_df.nsmallest(10, 'RTP')['Name']
fig = px.line(f_babynames[f_babynames["Name"].isin(top_declined_names)], 
              x="Year", y="Count", color="Name", 
              title="Top 10 Declined Names by RTP")
fig.update_layout(font_size=18, autosize=False, width=1000, height=400)
fig.show()
